In [1]:
!python -V

Python 3.9.15


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [9]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2024/05/21 23:04:36 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='/Users/weishanhe/my_github/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1716347076037, experiment_id='1', last_update_time=1716347076037, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [19]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [20]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [22]:
len(df_train), len(df_val)

(73908, 61921)

In [23]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [24]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [25]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715208946364

In [28]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [31]:
with mlflow.start_run():

    mlflow.set_tag("developer", "cristian")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:
import xgboost as xgb

In [33]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [34]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [35]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [36]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:8.77362                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:48:11] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[1]	validation-rmse:7.41847                           
[2]	validation-rmse:6.92566                           
[3]	validation-rmse:6.73735                           
[4]	validation-rmse:6.65530                           
[5]	validation-rmse:6.61400                           
[6]	validation-rmse:6.59376                           
[7]	validation-rmse:6.58409                           
[8]	validation-rmse:6.57972                           
[9]	validation-rmse:6.57148                           
[10]	validation-rmse:6.56788                          
[11]	validation-rmse:6.56165                          
[12]	validation-rmse:6.55783                          
[13]	validation-rmse:6.55430                          
[14]	validation-rmse:6.55148                          
[15]	validation-rmse:6.54394                          
[16]	validation-rmse:6.54034                          
[17]	validation-rmse:6.53686                          
[18]	validation-rmse:6.53114                          
[19]	valid

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [23:48:35] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.62635                                                    
[1]	validation-rmse:11.09542                                                    
[2]	validation-rmse:10.61662                                                    
[3]	validation-rmse:10.18503                                                    
[4]	validation-rmse:9.79628                                                     
[5]	validation-rmse:9.44875                                                     
[6]	validation-rmse:9.13577                                                     
[7]	validation-rmse:8.85573                                                     
[8]	validation-rmse:8.60896                                                     
[9]	validation-rmse:8.38698                                                     
[10]	validation-rmse:8.19095                                                    
[11]	validation-rmse:8.01326                                                    
[12]	validation-rmse:7.85999

...

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:19:50] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:7.35441                                                     
[1]	validation-rmse:6.66522                                                     
[2]	validation-rmse:6.54051                                                     
[3]	validation-rmse:6.49485                                                     
[4]	validation-rmse:6.47543                                                     
[5]	validation-rmse:6.46528                                                     
[6]	validation-rmse:6.45826                                                     
[7]	validation-rmse:6.45239                                                     
[8]	validation-rmse:6.44554                                                     
[9]	validation-rmse:6.44074                                                     
[10]	validation-rmse:6.43348                                                    
[11]	validation-rmse:6.43261                                                    
[12]	validation-rmse:6.43083

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [00:20:00] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.10595                                                    
[1]	validation-rmse:10.20216                                                    
[2]	validation-rmse:9.46894                                                     
[3]	validation-rmse:8.87834                                                     
[4]	validation-rmse:8.40753                                                     
[5]	validation-rmse:8.03202                                                     
[6]	validation-rmse:7.73733                                                     
[7]	validation-rmse:7.50137                                                     
[8]	validation-rmse:7.31847                                                     
[9]	validation-rmse:7.17722                                                     
[10]	validation-rmse:7.06060                                                    
[11]	validation-rmse:6.97109                                                    
[12]	validation-rmse:6.89774

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(



In [38]:
mlflow.xgboost.autolog(disable=True)

In [37]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:04:01] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:04:57] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [40]:
logged_model = 'runs:/95a9e0645b4548f4a5bac4a30539a699/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:12:18] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [41]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 95a9e0645b4548f4a5bac4a30539a699

In [42]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

/Users/weishanhe/opt/miniconda3/envs/mlops/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [22:13:39] WARNING: /Users/runner/work/xgboost/xgboost/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [43]:
xgboost_model

In [44]:
y_pred = xgboost_model.predict(valid)

In [45]:
y_pred[:10]

array([14.782765 ,  7.184751 , 15.971323 , 24.328938 ,  9.559302 ,
       17.115105 , 11.6522455,  8.688133 ,  8.962229 , 18.982166 ],
      dtype=float32)

In [46]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [54]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids = '1',
    filter_string = "metrics.rmse < 6.8",
    run_view_type = ViewType.ACTIVE_ONLY,
    max_results = 5,
    order_by = ["metrics.rmse ASC"]
)

In [55]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: d786efd9650646dd8ff936eaf5b8c9fe, rmse: 6.3095
run id: 1b235c38732f408d8864e750bc02fb11, rmse: 6.3104
run id: aeec77ccda164cec98b0b60639db3aa5, rmse: 6.3118
run id: b1f3eab4b34a4cb0abb1f61400eb67a1, rmse: 6.3138
run id: 0c99ae2f785e48ee8faa877664ed11c0, rmse: 6.3144


In [56]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [58]:
run_id = "d786efd9650646dd8ff936eaf5b8c9fe"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri = model_uri, name = "nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716606852607, current_stage='None', description=None, last_updated_timestamp=1716606852607, name='nyc-taxi-regressor', run_id='d786efd9650646dd8ff936eaf5b8c9fe', run_link=None, source='/Users/weishanhe/my_github/mlops-zoomcamp/02-experiment-tracking/mlruns/1/d786efd9650646dd8ff936eaf5b8c9fe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [61]:
# transition model to production
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name = model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: None


/var/folders/85/ncp1phfs6932692zx4c80w7c0000gn/T/ipykernel_94274/2919905036.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name = model_name)


In [63]:
model_version = 3
new_stage = "Staging"
client.transition_model_version_stage(
    name = model_name,
    version = model_version,
    stage = new_stage,
    archive_existing_versions = False
)

/var/folders/85/ncp1phfs6932692zx4c80w7c0000gn/T/ipykernel_94274/3649399860.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1716606852607, current_stage='Staging', description=None, last_updated_timestamp=1716607290565, name='nyc-taxi-regressor', run_id='d786efd9650646dd8ff936eaf5b8c9fe', run_link=None, source='/Users/weishanhe/my_github/mlops-zoomcamp/02-experiment-tracking/mlruns/1/d786efd9650646dd8ff936eaf5b8c9fe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [64]:
from datetime import datetime
date = datetime.today().strftime('%Y-%m-%d')
client.update_model_version(
    name = model_name,
    version = 3,
    description = f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1716606852607, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2024-05-24', last_updated_timestamp=1716607359386, name='nyc-taxi-regressor', run_id='d786efd9650646dd8ff936eaf5b8c9fe', run_link=None, source='/Users/weishanhe/my_github/mlops-zoomcamp/02-experiment-tracking/mlruns/1/d786efd9650646dd8ff936eaf5b8c9fe/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>